## Exemple du fonctionnement de l'algorithme de chiffrement utilisé par SEAL
* SEAL est une librairie de chiffrement homomorphe par Microsoft

https://eprint.iacr.org/2017/224.pdf

* SEAL est basé sur le chiffrement BFV

https://eprint.iacr.org/2012/144.pdf

* Code inspiré par

https://arxiv.org/pdf/1906.07127.pdf

* Implémenté sur le framework Python SageMath

https://www.sagemath.org

In [1]:
from sage.stats.distributions.discrete_gaussian_integer import DiscreteGaussianDistributionIntegerSampler

In [2]:
# Modulus in the ciphertext space
q = 2^35 - 2^14 + 2^11 + 1
# Modulus in the plaintext space (doit etre au moins 2x plus grand que le nombre a representer)
t = 128
# A power of 2
n = 1024
# The polynomial modulus which specifies the ring R
f = x^n + 1
# The ring Z[x]/f(x)
P.<x> = PolynomialRing(ZZ)
R.<x> = QuotientRing(P, P.ideal(f))
# Error distribution (a truncated discrete Gaussian distribution)
noise_standard_deviation = 3.20
dist = DiscreteGaussianDistributionIntegerSampler(sigma=noise_standard_deviation)
# Quotient on division of q by t
delta = floor(q / t)

In [3]:
def rq(poly):
    coefs = poly.list()
    for i in range(len(coefs)):
        coefs[i] = coefs[i] % q
    return R(coefs)

def rt(poly):
    coefs = poly.list()
    for i in range(len(coefs)):
        coefs[i] = coefs[i] % t
    return R(coefs)

# Cyphertext Space
def Roundq(poly):
    coefs = poly.list()
    for i in range(len(coefs)):
        coefs[i] = coefs[i] % q
        if coefs[i] > q / 2:
            coefs[i] = coefs[i] - q
    return R(coefs)

# Plaintext Space 
def Roundt(poly):
    coefs = poly.list()
    for i in range(len(coefs)):
        coefs[i] = coefs[i] % t
        if coefs[i] > t / 2:
            coefs[i] = coefs[i] - t
    return R(coefs)

In [4]:
assert q % (2*n) == 1
assert delta * t + rt(q) == q

In [5]:
# Generate a random ternary polynomial (coefs are either -1, 0 or 1)
def sample_poly_ternary():
    return R([ZZ.random_element(-1, 2, "uniform") for _ in range(n)])

# Generate a random polynomial with integer coefs from a Gaussian Distribution χ
def sample_poly_normal():
    return R([dist() for _ in range(n)])
    
# Generate a random polynomial with coefs in Z/qZ
def sample_Rq():
    return R([ZZ.random_element(0, q, "uniform") for _ in range(n)])

## Génération de clé
* Dans SEAL, la clé privé est un polynome cyclotomique

In [6]:
def secretKeyGen():
    return sample_poly_ternary()

### Calcule de la clé publique correspondante
$$a \leftarrow R_q, e \leftarrow \chi$$
$$pk = \big(-[a*s+e]_q, a\big)$$

In [7]:
def publicKeyGen(s):
    a = sample_Rq()
    e = sample_poly_normal()
    return [-Roundq(a * s + e), a]

In [8]:
sk = secretKeyGen()
pk = publicKeyGen(sk)

## La fonction de chiffrement
$$c = \big([p_0 * u + e_1 + \Delta * m]_q, [p_1 * u + e_2]_q\big)$$

In [9]:
def encrypt(p, m):
    u = sample_poly_ternary()
    e1 = sample_poly_normal()
    e2 = sample_poly_normal()
    return [Roundq(p[0] * u + e1 + delta * m), Roundq(p[1] * u + e2)]

def decrypt(s, c):
    tmp = t * Roundq(c[0] + c[1] * s)
    coefs = tmp.list()
    for i in range(len(coefs)):
        coefs[i] = round(coefs[i] / q)
    return Roundt(R(coefs))

In [10]:
assert decrypt(sk, encrypt(pk, 1)) == 1

# Opérations entre message chiffrées
$(c_1,c_2) \in C$,
$$c_1 \oplus c_2$$
## Addition
$$\Big(\big[c_1[0] + c_2[0]\big]_q , \big[c_1[1] + c_2[1]\big]_q \Big)$$

In [11]:
def add(c1, c2):
    return [Roundq(c1[0] + c2[0]), Roundq(c1[1] + c2[1])]

### Test
Ajoutons 20 à 22

In [12]:
c1 = encrypt(pk, 20)
c2 = encrypt(pk, 22)
print(decrypt(sk, add(c1, c2)))

42


## TODO: la multiplication
L'algorithme de multiplication de BFV est beaucoup plus compliqué que l'addition.
Il se fait en plusieurs étapes et nécessite des changements de bases.

Voir chapitre 4 de [Somewhat Practical Fully Homomorphic Encryption](https://eprint.iacr.org/2012/144.pdf) 

# Opérations entre un message chiffrée et un message en clair
Très utilisé dans CryptoNets https://www.microsoft.com/en-us/research/wp-content/uploads/2016/04/CryptonetsTechReport.pdf

$c_1 \in C, m_2 \in M,$
$$c_1 \oplus m_2$$

* Intuitivement, on pourrait chiffrer le message $m_1$ avant d'appliquer l'opération mais ce n'est pas très performant
* Ce genre d'opération est utile dans un réseau de neurones, par exemple lors du produit des poids (non chiffrés) avec l'image de l'utilisateur (chiffrée)

## Addition
$$c_1[0] + m_2 * \Delta = [p_0 * u + e_1 + \Delta * (m_1 + m_2)]_q $$

In [13]:
def add_plain(c1, m2):
    return [Roundq(c1[0] + delta * m2), c1[1]]

J'ajoute 20 chiffré à 1

In [14]:
print(decrypt(sk, add_plain(c1, 1)))

21


## Multiplication
$$c_1 * m_2 = \big([p_0 * u' + e_1' + \Delta * m_1 * m_2]_q, [p_1*u'+e_2']_q\big)$$

In [15]:
def mult_plain(c1, m2):
    return [Roundq(c1[0] * m2), Roundq(c1[1] * m2)]

Je multiplie 20 chiffré par 2

In [16]:
print(decrypt(sk, mult_plain(c1, 2)))

40


## Fonction
On peut donc appliquer des fonctions à nos données

Par exemple on va appliquer $f(x) = 2x+2$ à 20 chiffré

In [17]:
def f(c):
    return add_plain(mult_plain(c, 2), 2)

In [18]:
print(decrypt(sk, f(c1)))

42


### Exemple d'une clé privée

In [19]:
print(sk)

x^1022 + x^1020 - x^1018 + x^1017 + x^1016 - x^1014 - x^1013 + x^1012 - x^1011 + x^1010 + x^1007 + x^1006 - x^1005 - x^1004 + x^1003 - x^1002 + x^1001 - x^1000 + x^999 - x^998 + x^997 + x^996 - x^995 + x^994 + x^993 + x^992 + x^991 + x^990 - x^989 - x^988 + x^987 + x^986 + x^985 - x^984 - x^983 - x^981 - x^979 - x^978 + x^977 + x^976 - x^975 - x^971 - x^969 - x^968 + x^965 + x^963 - x^962 + x^960 - x^959 - x^957 - x^956 - x^955 + x^954 + x^953 - x^951 + x^950 + x^949 - x^947 - x^946 + x^945 - x^944 + x^943 - x^942 + x^941 + x^939 - x^935 + x^934 - x^930 - x^927 + x^924 - x^923 - x^922 - x^921 - x^917 - x^916 - x^915 - x^913 + x^912 - x^910 + x^909 + x^908 - x^907 - x^906 + x^905 + x^904 - x^903 - x^902 + x^901 - x^900 - x^899 + x^897 - x^896 + x^894 - x^893 + x^892 + x^891 + x^889 + x^888 - x^885 - x^882 + x^881 + x^880 + x^878 - x^877 + x^876 - x^875 + x^874 + x^870 + x^869 + x^867 + x^866 + x^865 - x^864 + x^862 - x^861 + x^860 + x^858 - x^856 + x^855 + x^854 + x^850 + x^849 + x^848 

### Exemple d'un nombre chiffré

In [20]:
print(c1)

[-11425718130*x^1023 - 4403920575*x^1022 + 8653998140*x^1021 + 15620674161*x^1020 + 10610571732*x^1019 - 12402248220*x^1018 - 12801808272*x^1017 + 2282424072*x^1016 - 13056153525*x^1015 - 2067942570*x^1014 + 7424887003*x^1013 + 8240605065*x^1012 - 8105575718*x^1011 - 8975213868*x^1010 - 3020716518*x^1009 - 14040024181*x^1008 - 16875581142*x^1007 - 11161522006*x^1006 + 11136250960*x^1005 + 7271666334*x^1004 + 6215628079*x^1003 + 9621705108*x^1002 + 3814733096*x^1001 - 2069547894*x^1000 - 15941645707*x^999 + 9620195070*x^998 + 12043532624*x^997 - 15381153300*x^996 - 16809986419*x^995 - 9778527897*x^994 + 2686748803*x^993 - 9491005373*x^992 + 9829423179*x^991 + 14645385020*x^990 - 3321015876*x^989 - 14501237472*x^988 + 4067244549*x^987 + 7001945960*x^986 - 13272068115*x^985 - 5638451057*x^984 - 7837359660*x^983 - 4665228945*x^982 - 1886428363*x^981 - 6966933080*x^980 - 5555721668*x^979 - 12080607450*x^978 + 9078319851*x^977 - 15966302106*x^976 + 11141816955*x^975 - 8691861080*x^974 + 9398